# *New York, New York*:
## Trashy Trends in New York City, 2005-2023
Pete Smith

Firstly, there isn't really much that can be done without the use of libraries. Or, I guess you *could* do everything without libraries, but why? People have already made these libraries for a reason, so let's use them.

In [94]:
import pandas as pd
import numpy as np

Now comes the time to load in the data. I pre-downloaded the data for convenience's sake. It is available at the following link:

    https://catalog.data.gov/dataset/dsny-waste-characterization-comparative-results

Immediately displaying the table will help to identify certain issues with the data that will need to be addressed later.

For the sake of saving space, I will only use `df.head(5)` here. However, larger ranges can be used instead or entirely different row selections can be used.

In [95]:
df = pd.read_csv("./DSNY_Waste_Characterization_-_Comparative_Results.csv")

df.head(5)

,Year,Material Group,Comparative Category,DSNY Diversion Summary Category,Aggregate Percent,Refuse Percent,"MGP (Metal, Glass, Plastic) Percent",Paper Percent,Organics Percent,"Generator (Residential, Schools, NYCHA)"
0,2023,Paper,Newspaper,Designated Paper,0.8%,0.005,0.4%,4.8%,0.0%,Residential
1,2023,Paper,Plain OCC/Kraft Paper,Designated Paper,7.2%,0.022,2.8%,56.6%,0.2%,Residential
2,2023,Paper,High Grade Paper,Designated Paper,0.4%,0.003,0.1%,1.3%,0.0%,Residential
3,2023,Paper,Mixed Low Grade Paper,Designated Paper,7.4%,0.062,3.3%,22.5%,0.2%,Residential
4,2023,Paper,Paper: Compostable/Soiled/Waxed OCC/Kraft,Organics Suitable for Composting,9.0%,0.105,2.4%,2.9%,0.8%,Residential


Upon review different parts of the table with different techniques, as discussed above. Unknown values appear to be denoted as '-' in the table. This will be... unfortunate... if it isn't fixed because '-' cannot be cast to an integer, float, or other number-like type. Mixing numbers and strings would make things exceedingly difficult going forward, so at this point '-' needs to be replaced with Numpy's NaN value.

As a way of showing this, I have included summing the "Aggregate Percent" column in the CSV both before and after the '-' was changed to NaN.

In [96]:
try:
    df["Aggregate Percent"] = df["Aggregate Percent"].str.rstrip("%").astype("float")
    total = df["Aggregate Percent"].sum()
except:
    print("It doesn't work before changing '-' to NaN")

df.replace("-", np.nan, inplace=True)

df["Aggregate Percent"] = df["Aggregate Percent"].str.rstrip("%").astype("float")
total = df["Aggregate Percent"].sum()
print("No error after changing '-' to NaN")

df.head(5)

It doesn't work before changing '-' to NaN
No error after changing '-' to NaN


,Year,Material Group,Comparative Category,DSNY Diversion Summary Category,Aggregate Percent,Refuse Percent,"MGP (Metal, Glass, Plastic) Percent",Paper Percent,Organics Percent,"Generator (Residential, Schools, NYCHA)"
0,2023,Paper,Newspaper,Designated Paper,0.8,0.005,0.4%,4.8%,0.0%,Residential
1,2023,Paper,Plain OCC/Kraft Paper,Designated Paper,7.2,0.022,2.8%,56.6%,0.2%,Residential
2,2023,Paper,High Grade Paper,Designated Paper,0.4,0.003,0.1%,1.3%,0.0%,Residential
3,2023,Paper,Mixed Low Grade Paper,Designated Paper,7.4,0.062,3.3%,22.5%,0.2%,Residential
4,2023,Paper,Paper: Compostable/Soiled/Waxed OCC/Kraft,Organics Suitable for Composting,9.0,0.105,2.4%,2.9%,0.8%,Residential


At this point, it might also be helpful to remove the percent signs from all of the other columns that have them. This can be done more or less the same as the example from before.

Additionally, the "Refuse Percent" column appears to be formatted as the decimal value rather than the percent format. E.g., instead of %20, it would say 0.20. This needs to be standardized as well, which is just a matter of multiplying it by 100 to match its scale with the rest of the percent columns.

In [97]:
df["MGP (Metal, Glass, Plastic) Percent"] = df["MGP (Metal, Glass, Plastic) Percent"].str.rstrip("%").astype("float")
df["Paper Percent"] = df["Paper Percent"].str.rstrip("%").astype("float")
df["Organics Percent"] = df["Organics Percent"].str.rstrip("%").astype("float")

df["Refuse Percent"] = df["Refuse Percent"] * 100

df.head(5)

,Year,Material Group,Comparative Category,DSNY Diversion Summary Category,Aggregate Percent,Refuse Percent,"MGP (Metal, Glass, Plastic) Percent",Paper Percent,Organics Percent,"Generator (Residential, Schools, NYCHA)"
0,2023,Paper,Newspaper,Designated Paper,0.8,0.5,0.4,4.8,0.0,Residential
1,2023,Paper,Plain OCC/Kraft Paper,Designated Paper,7.2,2.2,2.8,56.6,0.2,Residential
2,2023,Paper,High Grade Paper,Designated Paper,0.4,0.3,0.1,1.3,0.0,Residential
3,2023,Paper,Mixed Low Grade Paper,Designated Paper,7.4,6.2,3.3,22.5,0.2,Residential
4,2023,Paper,Paper: Compostable/Soiled/Waxed OCC/Kraft,Organics Suitable for Composting,9.0,10.5,2.4,2.9,0.8,Residential


Lastly regarding cleaning the data, there is a decent amount of data that I simply will not need. The primary culprit of this is waste categorization - there are currently three columns for cateogrizing types of waste. While each has a different level of specificity and a different purpose, the only one that is likely to be needed in this project is the primary category, or the "Material Group" column.

An unintended side effect of this is also that all of the columns fit on my screen. While this may change depending on what a given person's screen size is, it is an important consideration to make. Even if you can scroll horizontally, it can be very helpful to have a table that is small enough to fit all of the columns on one screen without scrolling.

In [98]:
df.drop(columns=["Comparative Category", "DSNY Diversion Summary Category"], inplace=True)

df.head(5)

,Year,Material Group,Aggregate Percent,Refuse Percent,"MGP (Metal, Glass, Plastic) Percent",Paper Percent,Organics Percent,"Generator (Residential, Schools, NYCHA)"
0,2023,Paper,0.8,0.5,0.4,4.8,0.0,Residential
1,2023,Paper,7.2,2.2,2.8,56.6,0.2,Residential
2,2023,Paper,0.4,0.3,0.1,1.3,0.0,Residential
3,2023,Paper,7.4,6.2,3.3,22.5,0.2,Residential
4,2023,Paper,9.0,10.5,2.4,2.9,0.8,Residential
